In [1]:
import csv
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

In [3]:
data2015 = pd.read_csv('marathon_results_2015.csv')

In [9]:
data2015.columns

Index(['Unnamed: 0', 'Bib', 'Name', 'Age', 'M/F', 'City', 'State', 'Country',
       'Citizen', 'Unnamed: 9', '5K', '10K', '15K', '20K', 'Half', '25K',
       '30K', '35K', '40K', 'Pace', 'Proj Time', 'Official Time', 'Overall',
       'Gender', 'Division'],
      dtype='object')

In [14]:
data2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26598 entries, 0 to 26597
Data columns (total 25 columns):
Unnamed: 0       26598 non-null int64
Bib              26598 non-null object
Name             26598 non-null object
Age              26598 non-null int64
M/F              26598 non-null object
City             26598 non-null object
State            24047 non-null object
Country          26598 non-null object
Citizen          1064 non-null object
Unnamed: 9       67 non-null object
5K               26598 non-null object
10K              26598 non-null object
15K              26598 non-null object
20K              26598 non-null object
Half             26598 non-null object
25K              26598 non-null object
30K              26598 non-null object
35K              26598 non-null object
40K              26598 non-null object
Pace             26598 non-null object
Proj Time        26598 non-null object
Official Time    26598 non-null object
Overall          26598 non-null int64
G

In [16]:
timeData2015 = data2015[['Bib','Age','M/F','Half', 'Official Time']]

In [18]:
timeData2015 = timeData2015[ ~ timeData2015['Half'].str.contains('-')]

In [19]:
timeData2015 = timeData2015[ ~ timeData2015['Official Time'].str.contains('-')]

In [39]:
timeData2015['Official Time']= pd.to_timedelta(timeData2015['Official Time'])
timeData2015['Half']= pd.to_timedelta(timeData2015['Half'])

In [41]:
timeData2015.sample(5)

,Bib,Age,M/F,Half,Official Time
17349,22676,65,M,01:54:00,03:53:32
9492,8962,42,M,01:41:50,03:28:15
13011,11350,46,M,01:48:21,03:38:48
25325,29594,28,F,02:23:58,05:10:15
15018,21893,51,F,01:49:51,03:45:15


In [42]:
timeData2015['Second Half']=timeData2015['Official Time']-timeData2015['Half']

In [43]:
timeData2015.sample(5)

,Bib,Age,M/F,Half,Official Time,Second Half
25904,31539,23,F,02:25:22,05:27:59,03:02:37
13229,7574,49,M,01:35:48,03:39:27,02:03:39
21335,18674,35,F,02:01:02,04:15:50,02:14:48
17643,21171,44,F,01:51:08,03:54:37,02:03:29
20068,25114,58,F,02:02:23,04:07:08,02:04:45
